###DataLake (Deltalake) + Lakehouse (Deltatables) - using Delta format (parquet+snappy+delta log)

Delta Lake is an open-source storage framework that brings reliability, ACID transactions, and performance to data lakes. It sits on top of Parquet files and is most commonly used with Apache Spark and Databricks.<br>
Delta Lake is the core storage layer behind Bronze–Silver–Gold (medallion) architectures.
<img src="https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-logo-whitebackground.png" style="width:300px; float: right"/>

## ![](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Creating our first Delta Lake table

Delta is the default file and table format using Databricks.


1. Delta Lake | Iceberg | Parquet<br>
This is the physical data layer where data actually lives (usually on cloud storage like S3, ADLS, GCS).<br>
Parquet that add:<br>
ACID transactions<br>
Time travel:- You can go back in time and see old data.<br>
Schema enforcement:- Delta Lake prevents writing data that does not match the table schema<br>
Incremental processing:- Process only new or changed data instead of reprocessing the entire dataset<br>
----
2. Unity Catalog:- who can access what data.<br>
This is the central governance and metadata layer.<br>
---
3. Lakehouse Core<br>
Lakehouse<bt>
This is the unifying concept:
One system for batch, streaming, BI, ML, AI<br>

Instead of:<br>
Data Lake (raw)
Data Warehouse (analytics)<br>
Feature store (ML)<br>
Separate BI store<br>
You have one platform.<br>
![](https://docs.databricks.com/aws/en/assets/images/well-architected-lakehouse-7d7b521addc268ac8b3d597bafa8cae9.png)
---
Lakehouse-Specific Pillars<br>
These make Lakehouse unique:<br>
Data Governance<br>
- Unity Catalog<br>
- Fine-grained permissions<br>
- Lineage<br>

Interoperability & Usability<br>
- Works with open formats<br>
- SQL, Python, Scala<br>
- No vendor lock-in<br>


In [0]:
%sql
drop table lakehousecat.deltadb.drugstb1;
drop table lakehousecat.deltadb.drugstbl_merge;

In [0]:
spark.sql("CREATE CATALOG IF NOT EXISTS lakehousecat")
spark.sql("CREATE SCHEMA IF NOT EXISTS lakehousecat.deltadb")
spark.sql("CREATE VOLUME IF NOT EXISTS lakehousecat.deltadb.datalake")

####1. Write data into delta file (Datalake) and table (Lakehouse)

In [0]:
df=spark.read.csv("/Volumes/lakehousecat/deltadb/datalake/druginfo.csv",header=True,inferSchema=True)#Reading normal data from datalake
df.write.format("delta").mode("overwrite").save("/Volumes/lakehousecat/deltadb/datalake/deltalakedir")#writing normal data from deltalake(datalake)
df.write.format("parquet").mode("overwrite").save("/Volumes/lakehousecat/deltadb/datalake/parquestlakedir")#writing normal data from parquet(datalake)

df.write.option("mergeSchema",True).saveAsTable("lakehousecat.deltadb.drugstb1",mode='overwrite')#writing normal data from deltalakehouse(lakehouse)

#behind it stores the data in deltafile format in the s3 bucket (location is hidden for us in databricks free edition)

In [0]:
#We can add Schema evolution feature just by adding the below option in Delta tables.
#df.write.option("mergeSchema","True").saveAsTable("lakehousecat.deltadb.drugstbl",mode='overwrite')

####2. DML Operations in Delta Tables & Files
Support for DELETE/UPDATE/MERGE

In [0]:
%sql
USE lakehousecat.deltadb

In [0]:
%sql
DESC HISTORY drugstb1

In [0]:
%sql
SELECT * FROM drugstb1

In [0]:
%sql
SELECT * FROM drugstb1
WHERE uniqueid=163740;

#####a. Table Update

In [0]:
%sql
UPDATE drugstb1
SET rating=rating-1
WHERE uniqueid=163740;

In [0]:
%sql
SELECT * FROM drugstb1
where uniqueid=163740;

#####b. Table Delete

In [0]:
%sql
DELETE FROM drugstb1
WHERE uniqueid=163740;

In [0]:
%sql
SELECT * FROM drugstb1
where uniqueid in (163740,206473);

In [0]:
%sql
DELETE FROM drugstb1
WHERE uniqueid in (66736,4907,97013)

In [0]:
%sql
DESC HISTORY drugstb1

#####c. File DML (Update/Delete)
We don't do file DML usually, we are doing here just for learning about 
- file also can be undergone with limited DML operation
- we need to learn about how the background delta operation is happening when i do DML

In [0]:
spark.read.format("delta").load("/Volumes/lakehousecat/deltadb/datalake/deltalakedir").where("uniqueid==163740").show()

In [0]:
#DML on Files: How to update delta files
from delta.tables import DeltaTable #Imports the Delta Lake API that allows you to perform DML operations programmatically (update, delete, merge) using PySpark.
deltafile = DeltaTable.forPath(spark,"/Volumes/lakehousecat/deltadb/datalake/deltalakedir")#This is called a path-based Delta table (not registered in Unity Catalog / Hive metastore).
deltafile.update("uniqueid=163740", { "rating": "rating - 1" } )

In [0]:
spark.read.format("delta").load("/Volumes/lakehousecat/deltadb/datalake/deltalakedir").where("uniqueid==163740").show()

#####d. File Delete


In [0]:
df=spark.read.format("delta").load('/Volumes/lakehousecat/deltadb/datalake/deltalakedir')
df.where('uniqueid=206473').show()

In [0]:
from delta.tables import DeltaTable
deltaTable = DeltaTable.forPath(spark, "/Volumes/lakehousecat/deltadb/datalake/deltalakedir")
deltaTable.delete("uniqueid=206473")

In [0]:
df=spark.read.format("delta").load('/Volumes/lakehousecat/deltadb/datalake/deltalakedir')
df.where('uniqueid=206473').show()

#####d. Merge Operation

In [0]:
%sql
select count(*) from drugstb1

In [0]:
%sql
CREATE OR REPLACE TABLE drugstbl_merge 
AS SELECT * FROM drugstb1 
WHERE rating<=8;

In [0]:
%sql
select count(*) from drugstbl_merge;

In [0]:
%sql
--Delta table support merge operation for (insert/update/delete)
--2899 updated
--2801 inserted
MERGE INTO drugstbl_merge tgt--2899
USING drugstb1 src--5700
ON tgt.uniqueid = src.uniqueid
WHEN MATCHED THEN
  UPDATE SET tgt.usefulcount= src.usefulcount,
             tgt.drugname = src.drugname,
             tgt.condition = src.condition
WHEN NOT MATCHED
  THEN INSERT (uniqueid,rating,date,usefulcount, drugname, condition ) VALUES (uniqueid,rating,date,usefulcount, drugname, condition);
  --5700-2899 = 2801

In [0]:
%sql
SELECT count(*) from drugstbl_merge

In [0]:
%sql
--After the below insert, Again try the merge in the above cell
insert into drugstbl_merge select 99999999,drugname,condition,rating,date,usefulcount 
from drugstb1 limit 1;

In [0]:
%sql
--Delta table support merge operation for (delete)
--1 deleted (which is not present in the source (source system deleted it already, hence target also has to delete))
MERGE INTO drugstbl_merge tgt
USING drugstb1 src
ON tgt.uniqueid = src.uniqueid
WHEN MATCHED THEN
UPDATE SET tgt.usefulcount= src.usefulcount,
          tgt.drugname = src.drugname,
          tgt.condition = src.condition
WHEN NOT MATCHED THEN INSERT(uniqueid,rating,date,usefulcount, drugname, condition) VALUES (uniqueid,rating,date,usefulcount, drugname, condition)
WHEN NOT MATCHED BY SOURCE
THEN DELETE 

-- When will DELETE happen?
-- A row in target table (drugstbl_merge = tgt) will be deleted IF AND ONLY IF:
-- The row exists in the target (tgt)
-- No matching row exists in the source (src)
-- Match condition fails:If a record exists in the target table but the same uniqueid does not exist in the source table, delete it from the target.”

%md
MATCHED<br>
NOT MATCHED (by target)<br>
NOT MATCHED BY SOURCE<br>

Case 1️⃣ — MATCHED-Row exists in BOTH source and target<br>
➡ Executes:WHEN MATCHED THEN UPDATE<Br>

Case 2️⃣ — NOT MATCHED (by target)<br>
Row exists in SOURCE<br>
but does NOT exist in TARGET<br>
➡ Executes:WHEN NOT MATCHED THEN INSERT<br>

Case 3️⃣ — NOT MATCHED BY SOURCE<br>
Row exists in TARGET<br>
but does NOT exist in SOURCE<br>
➡ Executes:WHEN NOT MATCHED BY SOURCE THEN DELETE<br>


Source table (drugstb1)
| uniqueid | drugname |
| -------- | -------- |
| 1        | A        |
| 2        | B        |
| 3        | C        |

drugstbl_merge
| uniqueid | drugname |
| -------- | -------- |
| 1        | A        |
| 2        | B        |
| 3        | C        |
| 4        | D        |

MERGE execution

uniqueid = 1 → MATCHED → UPDATE

uniqueid = 2 → MATCHED → UPDATE

uniqueid = 3 → MATCHED → UPDATE

uniqueid = 4 → NOT MATCHED BY SOURCE → DELETE


In [0]:
%sql
select count(*) from drugstbl_merge;

In [0]:
#Merge operation using spark with (library delta.tables.DeltaTable) DSL (not by using SQL) - SQL is better to use
from delta.tables import DeltaTable
print(spark.read.table("drugstb1").count())
print(spark.read.table("drugstbl_merge").count())
tgt = DeltaTable.forName(spark, "drugstbl_merge")
src = spark.table("drugstb1")
(
    tgt.alias("tgt")
    .merge(
        src.alias("src"),
        "tgt.uniqueid = src.uniqueid"
    )
    .whenMatchedUpdate(set={
        "usefulcount": "src.usefulcount",
        "drugname": "src.drugname",
        "condition": "src.condition"
    })
    .whenNotMatchedInsert(values={
        "uniqueid": "src.uniqueid",
        "rating": "src.rating",
        "date": "src.date",
        "usefulcount": "src.usefulcount",
        "drugname": "src.drugname",
        "condition": "src.condition"
    })
    .whenNotMatchedBySourceDelete()
    .execute()
)


In [0]:
print(spark.read.table("drugstb1").count())
print(spark.read.table("drugstbl_merge").count())

####3. Additional Operations on Deltalake & Deltatables

#####a. History & Versioning
*History* returns one row per commit/version and tells you what changed, when, and how.

In [0]:
%sql
DESC HISTORY drugstbl_merge

*Version as of* will reads the snapshot of drugstbl_merge at version 4 and Ignores all changes made in versions 5, 6, … current

In [0]:
%sql
SELECT count(*) from drugstbl_merge VERSION AS OF 3

#####b. Time Travel
*Timestamp as of* Reads the table as it existed at that exact timestamp and Any commits after the given timestamp is ignored

In [0]:
%sql
select count(1) from drugstbl_merge timestamp as of '2026-01-30T16:10:40.000+00:00'

#####c. Vaccum
*VACUUM* in Delta Lake removes old, unused files to free up storage, default retention hours is 168. These files come from operations like DELETE, UPDATE, or MERGE and are kept temporarily so time-travel queries can work.

AI Suggested feature<br>
Setting the Delta table property 'delta.deletedFileRetentionDuration' to less than the default (1 week) is generally not recommended for production environments. Lowering the retention duration can lead to data loss if you need to time travel or restore data, as older files may be deleted sooner than expected. The default of 168 hours (1 week) is chosen to balance storage costs and safety for production workloads. Only reduce this value if you fully understand the risks and have a strong operational reason to do so

In [0]:
%sql
--These properties can't be set in serverless, we will see this in cluster or i will get a table with more than 1 week data
--SET spark.databricks.delta.retentionDurationCheck.enabled = false;
--alter table drugstbl_merge SET TBLPROPERTIES ('delta.deletedFileRetentionDuration' = '24 hours');

In [0]:
%sql
--default 168 hours (1 week), less than 1 week will not work in serverless for performance and session state reason
VACUUM drugstbl_merge RETAIN 168 HOURS;

-- You are keeping 7 days of history
-- Time travel is possible up to 7 days back
-- Anything older is permanently deleted

In [0]:
%sql
DESC HISTORY drugstbl_merge

In [0]:
%sql
SELECT count(1) FROM drugstbl_merge TIMESTAMP AS OF '2026-01-30T16:10:37.000+00:00';

In [0]:
spark.sql("VACUUM drugstbl_merge RETAIN 168 HOURS")

In [0]:
%sql
DESC HISTORY drugstbl_merge

#####d. ACID Transactions
**Delta Lake uses ACID transactions under the hood via a transaction log.**
| ACID        | In Databricks         |
| ----------- | --------------------- |
| Atomicity   | All or nothing or Individual Transactions |
| Consistency | Schema + constraints  |
| Isolation   | Snapshot isolation    |
| Durability  | Transaction log       |

In [0]:
%sql
CREATE OR REPLACE TABLE acid_demo_txn (
  id INT,
  amount INT
)USING DELTA;


In [0]:
%sql
select * from acid_demo_txn;

In [0]:
%sql
--All or nothing (Atomic/Individual Transaction)
INSERT INTO acid_demo_txn VALUES
(1, 100),
(2, 200),
(3, 300);

In [0]:
%sql
DELETE FROM acid_demo_txn

In [0]:
%sql
--All or nothing (Atomic/Individual Transaction)
INSERT INTO acid_demo_txn VALUES
(1, 100),
(2, 200),
(3, 300);
-- (3, '300');

In [0]:
%sql
select * from acid_demo_txn;

In [0]:
%sql
select * from acid_demo_txn where id=1;

In [0]:
%sql
--Atomicity (Individual transaction that doesn't affect the other)
UPDATE acid_demo_txn SET amount = amount + 100 WHERE id = 1;--individual/atomic
UPDATE acid_demo_txn SET amount = amount + 200 WHERE id = 1;--individual/atomic
describe history acid_demo_txn;

--START TRANSACTION; UPDATE acid_demo_txn SET amount = amount + 100 WHERE id = 1; commit;
--START TRANSACTION; UPDATE acid_demo_txn SET amount = amount + 200 WHERE id = 1; commit;

In [0]:
%sql
select * from acid_demo_txn where id=1;

In [0]:
%sql
--Apply constraint for maintaining consistancy
--We can apply in databricks deltatable, 2 types of constraints (check and not null), 
-- in other DBs we can use primary key, foreign key and unique constraints also..
ALTER TABLE acid_demo_txn
ADD CONSTRAINT positive_amount CHECK (amount > 0);
INSERT INTO acid_demo_txn VALUES (4, 100);--Atomicity and consistancy
--INSERT INTO acid_demo_txn VALUES (5, -100);--Atomicity and consistancy



In [0]:
%sql
ALTER TABLE acid_demo_txn
DROP CONSTRAINT positive_amount

In [0]:
%sql
select * from acid_demo_txn;

In [0]:
%sql
--Isolation (We can achieve using timetravel (restore operation (no rollback)))
--START TRANSACTION; SAVEPOINT before_delete; DELETE FROM employees WHERE employee_id = 129; ROLLBACK TO before_delete;
--Notebook1 (We can see the data in notebook1)
--BEGIN TRANSACTION;
UPDATE acid_demo_txn SET amount = 999 WHERE id = 2;--Serialized (Committed read)
--Notebook2 (We can see the data in notebook2 )
--Notebook2 (We can see the data in notebook2 )
use lakehousecat.deltadb;
select * from (select * from acid_demo_txn version as of 14) where id=2;--serializable read (after the data successfully committed)

In [0]:
%sql
DESC HISTORY acid_demo_txn

In [0]:
%sql
UPDATE acid_demo_txn
SET amount=1000
WHERE id=1 

In [0]:
%sql
DESC HISTORY acid_demo_txn

In [0]:
%sql
SELECT * FROM acid_demo_txn

In [0]:
%sql
--something like savepoint+rollback (but not really a rollback (TCL is not available in Databricks in the name of commit, rollback, savepoint))
restore table acid_demo_txn to version as of 41

In [0]:
%sql
SELECT * FROM acid_demo_txn VERSION AS OF 39;

#####e. Transactions Control using restore
Bigdata ecosystems such as spark/databricks/delta are not Transaction in nature, hence it will not support TCL

In [0]:
%sql
--select count(1) from deltadb.drugs where date>'2012-02-28';
--4329
delete from drugstb1 where date>'2012-02-28';


In [0]:
%sql 
select count(1) from drugstb1;

In [0]:
%sql
describe history drugstb1;

In [0]:
%sql
RESTORE TABLE drugstb1 TO VERSION AS OF 2;

In [0]:
%sql
select count(1) from drugstb1;

In [0]:
%sql
--We can restore to any older/later version (unlit 168 hours/vacumm period)
RESTORE TABLE drugstb1 TO VERSION AS OF 4;

In [0]:
%sql
select count(1) from drugstb1;